In [1]:
import os
import numpy as np
import scipy.ndimage
import skimage.io
import skimage.color
import skimage.exposure
import cv2

from tqdm import tqdm
from skimage.measure import block_reduce
from skimage import transform
from utility import *

# Operations to prepare the DAVIS Dataset

Using the DAVIS 2019 unsupervised test-dev dataset from https://davischallenge.org/davis2017/code.html.

The Notebook uses the given video-frames to:
1. create grayscale images
2. resizes the images to a given size
3. compute optical flow
4. (create playable videos from the images)



TODO:

  - image resolution:
    - Davis 2019: 854x480
    - We probably want to rescale to smaller images, e.g.: 256×144, 320×180, 400×222, 512×288, 568×320, 640×360 
    
    (https://en.wikipedia.org/wiki/Low-definition_television)
  - optical flow:
    - We might want to use a different optical flow method, e.g. DeepFlow
    - We should think how to best store the optical flow images (maybe 2-channel images)
    - want to convey pixel-2-pixel correspondence - 
      need store flow such that reconstruction of vectors is possible
    - IDEA:
      - set reasonable maximum flow value (e.g. 100 pixels - 256bit img = 256px?)
      - normalize flow values to [0,1] (divide by max)
      - store as:
        - 2-channel (x,y) 
        - 3-channel (r,g,b) image 
      - reconstruct flow vector by multiplying with max
      

## 1. Converting images to grayscale

In [2]:
def image_conversion(dataset_path, gray, resize, image_size, target_size, subdir):
    """
    The function takes images from the DAVIS dataset and convert them to the desired 
    size and color defined by the parameters.

    :param dataset_path: The path to the (DAVIS) dataset as a string
    :param gray: A boolean to decide if the output images should be in grayscale or not
    :param resize: A boolean to decide if the output images should be resized
    :param image_size: The input image size as a tuple
    :param target_size: The output image size as a tuple
    :param subdir: The name of the subdirectory of DAVIS as a string. whether 'train', 'val' or 'test'
    """
    
    if not resize: 
        target_size = image_size
    
    source_res = str(image_size[0]) + 'p'
    target_res = str(target_size[0])  + 'p'
    input_path = os.path.join(dataset_path, subdir, source_res)    

    output_path = os.path.join(dataset_path, subdir, target_res)
    if gray: 
        output_path = os.path.join(output_path + '_gray')

    make_dir(output_path)
    print('Input path: {}'.format(input_path))
    print('Output path: {}'.format(output_path))

    subdirs = os.listdir(input_path)
    
    for subdir in tqdm(subdirs):
        if subdir == '.DS_Store': continue
        subdir_path = os.path.join(input_path, subdir)
        if not os.path.exists(os.path.join(output_path, subdir)):
            os.makedirs(os.path.join(output_path, subdir))
        filenames = os.listdir(subdir_path)
        for filename in filenames:
            image_path = os.path.join(subdir_path, filename)
            image = skimage.io.imread(image_path)
            if gray:
                image = skimage.color.rgb2gray(image)
                image = skimage.exposure.rescale_intensity(image, out_range=(0, 255))
                image = image.astype(np.uint8)
            if resize:
                image = skimage.transform.resize(image, target_size, preserve_range=True)
            image = image.astype(np.uint8)
            skimage.io.imsave(os.path.join(output_path, subdir, filename), image)

In [5]:
# HYPERPARAMETERS
dataset_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS'
gray = True
resize = True
image_size = (480, 854)
target_size = (176, 320)
subdir = 'test'

# Run the actual function using aboves hyperparameters
image_conversion(dataset_path, gray, resize, image_size, target_size, subdir)

Input path: /Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/test/480p
Output path: /Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/test/176p_gray


100%|████████████████████████████████████████████████████████████████████████████████| 31/31 [01:16<00:00,  2.46s/it]


## 2. Computing optical flows between consecutive images

In [6]:
def dense_optical_flow(method, image_path, output_image_path, array_path, params, gray, save_array):
    """
    Optical flow implementation: Compute optical flow between consecutive frames 
    (code taken from: https://learnopencv2.com/optical-flow-in-opencv/).

    :param method: built-in cv2 method to compute the optical flow; name as a string. 
        current options: 'deepflow', 'farneback' or 'lucaskanade_dense'
    :param image_path: the path to all frames to one video
    :param output_image_path: the path where the flow images are stored (as .jpg)
    :param array_path: the path where the flow data is stored (as .npy)
    :param params: an array with specific parameters for farneback algorithm, 
        empty array when using a different algorithm
    :param gray: boolean whether the images should be grayscaled or not
    :param save_array: boolean to decide whether the .jpg images or the .npy data is stored
    """
    
    # Read the images
    image_names = os.listdir(image_path)
    image_names.sort()
    old_frame = cv2.imread(os.path.join(image_path, image_names[0]))

    # create HSV & make Saturation a constant
    hsv = np.zeros_like(old_frame)
    hsv[..., 1] = 255

    # Convert to grayscale
    if gray:
        old_frame = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    
    for i in range(1, len(image_names)):
        # Read the next frame
        new_frame = cv2.imread(os.path.join(image_path, image_names[i]))

        # Convert to grayscale
        if gray:
            new_frame = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)

        # Calculate dense optical flow by Farneback method
        flow = method(old_frame, new_frame, None, *params)

        if save_array:
            flow_ds = skimage.transform.resize(flow, target_size, preserve_range=True)
            np.save(os.path.join(array_path, image_names[i]), flow_ds)
        else:
            # Encoding: convert the algorithm's output into Polar coordinates
            mag, ang = cartToPol(flow[..., 0], flow[..., 1])
            # Use Hue and Value to encode the Optical Flow
            hsv[..., 0] = (ang+np.pi) * 180 / ( 2 * np.pi )
            hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            # Convert HSV to RGB (BGR) color representation
            bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

            # Save output image
            if resize:
                bgr = skimage.transform.resize(bgr, target_size, preserve_range=True)
            cv2.imwrite(os.path.join(output_image_path, image_names[i]), bgr)

        k = cv2.waitKey(25) & 0xFF
        if k == 27:
            break

        # Update previous frame
        old_frame = new_frame

In [7]:
def calculate_flow(method, dataset_path, subdir, flowdir, gray, image_size, target_size, save_array):
    """
    This function calculates the optical flow of all subdirectories of the subdir folder.
    It is the function to be executed when performing the optical flow of the train, test or val folders
    should be calculated.

    :param method: Built-in cv2 method to compute the optical flow; name as a string. 
        current options: 'deepflow', 'farneback' or 'lucaskanade_dense'
    :param dataset_path: The path to the (DAVIS) dataset as a string
    :param subdir: The name of the subdirectory of DAVIS as a string. whether 'train', 'val' or 'test'
    :param flowdir: The name of the directory where the flow data should be stored
    :param gray: boolean whether the images should be grayscaled or not
    :param image_size: The input image size as a tuple
    :param target_size: The output image size as a tuple
    :param save_array: boolean to decide whether the .jpg images or the .npy data is stored
    """
    
    # using original resolution images and donwsampling the calculated optical flow to target size
    source_res = str(image_size[0]) + 'p'
    target_res = str(target_size[0])  + 'p'
    input_path = os.path.join(dataset_path, subdir, source_res)
    if gray: 
        input_path = os.path.join(input_path + '_gray')
    
    print('Input path: {}'.format(input_path))
    subdirs = os.listdir(input_path)
    
    output_path = os.path.join(dataset_path, subdir, target_res + '_' + method)
    make_dir(output_path)
    print('Output path: {}'.format(output_path))
    
    array_path = os.path.join(dataset_path, subdir, flowdir, target_res + '_' + method)
    make_dir(array_path)
    print('Array path: {}'.format(array_path))
    
    params = []
    
    # Specifying the optical flow algorithm and applying it to the dataset
    if method == 'farneback':
        algorithm = cv2.calcOpticalFlowFarneback
        params = [0.5, 3, 15, 3, 5, 1.2, 0]
        # Farneback's algorithm parameters: pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags
        
    if method == 'lucaskanade_dense':
        algorithm = cv2.optflow.calcOpticalFlowSparseToDense
        
    if method == 'deepflow':
        deepflow = cv2.optflow.createOptFlow_DeepFlow()
        algorithm = deepflow.calc
        
    # Computing optical flows for each folder in the dataset:
    print('Calculating dense optical flow using ' + method + ' method...')
    for subdir in tqdm(subdirs):
        if subdir == '.DS_Store': continue
        image_path = os.path.join(input_path, subdir)
        output_image_path = os.path.join(output_path, subdir)
        output_array_path = os.path.join(array_path, subdir)
        make_dir(output_image_path)
        make_dir(output_array_path)
        dense_optical_flow(algorithm, image_path, output_image_path, output_array_path, params, gray, save_array)

In [9]:
# HYPERPARAMETERS
method = 'deepflow'
dataset_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS'
subdir = 'test'
flowdir = 'flow'
gray = True
image_size = (480, 854)
target_size = (176, 320)
save_array = True

# Run the actual function to calculate the optical flow
calculate_flow(method, dataset_path, subdir, flowdir, gray, image_size, target_size, save_array)

Input path: /Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/test/480p_gray
Output path: /Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/test/176p_deepflow
Array path: /Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS/test/flow/176p_deepflow
Calculating dense optical flow using deepflow method...


100%|█████████████████████████████████████████████████████████████████████████████| 30/30 [1:00:53<00:00, 121.79s/it]


## 3. Converting images to video

In [ ]:
def images_2_video(image_folder, video_name, fps=30):
    image_names = os.listdir(image_folder)
    image_names.sort()
    frame = cv2.imread(os.path.join(image_folder, image_names[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, fps, (width,height))

    for image in image_names:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [ ]:
def all_images_2_video(dataset_path, subdir, res, video_folder, fps):
    image_path = os.path.join(dataset_path, subdir, res)
    video_path = os.path.join(dataset_path, subdir, video_folder, res)

    make_dir(video_path)
    
    print('Image path: {}'.format(image_path))
    print('Video path: {}'.format(video_path))

    subdirs = os.listdir(image_path)
    print('Number of subdirectories: {}'.format(len(subdirs)))

    for subdir in tqdm(subdirs):
        if subdir == '.DS_Store': continue
        image_folder = os.path.join(image_path, subdir)
        video_name = os.path.join(video_path, subdir + '.avi')
        images_2_video(image_folder, video_name, fps)

In [ ]:
# HYPERPARAMETERS
dataset_path = '/Users/Mika/Desktop/10_Semester/3DCV/PROJECT/DAVIS'
subdir = 'train'
res = '480p'
video_folder = 'AVI_videos'
fps = 24

# Run the actual function to make the images of a complete folder to videos
# all_images_2_video(dataset_path, subdir, res, video_folder, fps)